In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 200)

In [2]:
def postprocessing(df, sheet_name):

    step = int(sheet_name.split('_')[1])

    #df = df.drop(['Percentage_common', 'highest_indices_pred', 'highest_indices_actual_1'], axis=1)

    len(df['Combination'].unique())
    
    # Step 1: Full aggregation for MSE, MAE, and Highest simulation in pred
    aggregated_df = df.groupby('Combination').agg({
        'MSE': 'mean',
        'MAE': 'mean',
        'Highest simulation in pred': lambda x: x.mean()  # Convert to percentage
    }).reset_index()

    df['difference'] = df['Index of highest simulation'] - (df['Simulations seen before'] - 1)

    # Step 2: Get the index of the highest simulation in pred - not highest index but highest difference!
    idx_max_diff = df.groupby('Combination')['difference'].idxmax()
    max_diff_df = df.loc[idx_max_diff, ['Combination', 'Index of highest simulation', 'Simulations seen before', 'difference']]
    max_diff_df.set_index('Combination', inplace=True)

    # Step 3: Merge the two results 
    final_result = aggregated_df.merge(max_diff_df, on='Combination')
    
    final_result = final_result.sort_values(by='Highest simulation in pred', ascending=False)
    final_result['Step_Value'] = step

    final_result['Exploitation Stepts required'] = np.ceil(final_result['difference'] / 3).astype(int) 
    final_result['Exploration + Exploitation Steps required'] = final_result['Step_Value'] + final_result['Exploitation Stepts required']

    return final_result[['Step_Value', 'Combination', 'MSE', 'MAE', 'Index of highest simulation', 'Simulations seen before', 'difference', 'Highest simulation in pred', 'Exploitation Stepts required', 'Exploration + Exploitation Steps required']]

# PERFORM POSTPROCESSING PER EXCEL
def wrapper(df):
    results = []
    for sheet_name in df.keys():
        results.append(postprocessing(df[sheet_name], sheet_name))
    combined_df = pd.concat(results, ignore_index=True)
    return combined_df

In [6]:
# COMBINE THE RESULTS FROM THE EXCEL FILES
#files = ['Excels/SVR_RS_model_results.xlsx'] # 'Excels/SVR_EX_model_results.xlsx', 
#files = ['Excels/GP_model_results_MATERN.xlsx', 'Excels/GP_model_results_RBF_LINEAR.xlsx', 'Excels/GP_model_results_PERIODIC.xlsx']
files = ['../runs_06-13 11:22/GP_model_results.xlsx']

result_list = []
original_list = []
for file in files:
    print(file)
    df = pd.read_excel(file, sheet_name=None)
    result_list.append(wrapper(df))
    original_list.append(pd.concat(df.values(), ignore_index=True))

combined_results = pd.concat(result_list, axis=0)
combined_results.sort_values(by='Highest simulation in pred', ascending=False, inplace=True)
combined_original = pd.concat(original_list, axis=0)

../runs_06-13 11:22/GP_model_results.xlsx


In [7]:
# WRITE RESULTS TO EXCEL
with pd.ExcelWriter(files[0].split('/')[1].split('_')[0] + '_postprocessed.xlsx', engine='xlsxwriter') as writer:
    combined_results.to_excel(writer, sheet_name='Postprocessed', index=False)
    combined_original.to_excel(writer, sheet_name='Original', index=False)

# DOUBLECHECKS

In [8]:
# CHECKING FUNCTIONALITY
data = {
    'Combination': ['A', 'A', 'A', 'B', 'B'],
    'MSE': [0.1, 0.2, 0.3, 0.4, 0.5],
    'MAE': [0.01, 0.02, 0.03, 0.04, 0.05],
    'Index of highest simulation': [10, 20, 30, 40, 50],
    'Simulations seen before': [1, 2, 3, 4, 5],
    'Highest simulation in pred': [True, False, True, False, True]
}
df = pd.DataFrame(data)

pdf = postprocessing(df, 'Step_1')
pdf

,Step_Value,Combination,MSE,MAE,Index of highest simulation,Simulations seen before,difference,Highest simulation in pred,Exploitation Stepts required,Exploration + Exploitation Steps required
0,1,A,0.20,0.020,30,3,28,0.666667,10,11
1,1,B,0.45,0.045,50,5,46,0.500000,16,17


In [9]:
combined_results.sort_values(by='Highest simulation in pred', ascending=False).head(20)

,Step_Value,Combination,MSE,MAE,Index of highest simulation,Simulations seen before,difference,Highest simulation in pred,Exploitation Stepts required,Exploration + Exploitation Steps required
10,11,lr_0.1_k_Matern_lp_None_ls_0.01_lm_1.0_np_None...,0.002211,0.026469,1,2,0,1.000000,0,11
12,13,lr_0.1_k_Matern_lp_None_ls_0.01_lm_1.0_np_None...,0.001980,0.024780,1,2,0,1.000000,0,13
6,7,lr_0.1_k_Matern_lp_None_ls_0.01_lm_1.0_np_None...,0.002693,0.031709,2,2,1,1.000000,1,8
7,8,lr_0.1_k_Matern_lp_None_ls_0.01_lm_1.0_np_None...,0.002700,0.031091,2,3,0,1.000000,0,8
8,9,lr_0.1_k_Matern_lp_None_ls_0.01_lm_1.0_np_None...,0.002556,0.029582,1,2,0,1.000000,0,9
11,12,lr_0.1_k_Matern_lp_None_ls_0.01_lm_1.0_np_None...,0.002119,0.025649,1,2,0,1.000000,0,12
19,20,lr_0.1_k_Matern_lp_None_ls_0.01_lm_1.0_np_None...,0.001291,0.019880,1,2,0,1.000000,0,20
9,10,lr_0.1_k_Matern_lp_None_ls_0.01_lm_1.0_np_None...,0.002379,0.027610,1,2,0,1.000000,0,10
13,14,lr_0.1_k_Matern_lp_None_ls_0.01_lm_1.0_np_None...,0.001454,0.022866,1,2,0,1.000000,0,14
14,15,lr_0.1_k_Matern_lp_None_ls_0.01_lm_1.0_np_None...,0.001383,0.022195,1,2,0,1.000000,0,15


In [10]:
combined_results.value_counts('Step_Value').sort_values(ascending=False)

Step_Value
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    1
Name: count, dtype: int64

In [11]:
combined_results.value_counts('Combination').sort_values(ascending=False)

Combination
lr_0.1_k_Matern_lp_None_ls_0.01_lm_1.0_np_None_ns_0.1_nm_1.0_nc_1e-06_lt_ARD_af_UCB_rl_0.001    20
Name: count, dtype: int64